In [ ]:
dataframes = []
filename = ['alcohol','smokers','lifeexp65','socsupport']
for i in range(len(filename)):
    temp_df = pd.read_csv(f"{filename[i]}.csv")
    dataframes.append(temp_df)

alcohol = dataframes[0]
smokers = dataframes[1]
life_exp_65 = dataframes[2]
social_support = dataframes[3]

In [ ]:
### Det her er de enkelte figurer og noget (i toppen) som jeg ikke helt ved hvad er, og ikke tror vi skal bruge

alcohol_wide = pd.pivot(alcohol,index='country',columns='year',values='alcohol_sale')

col_dict = {}
for i in range(1960,2022):
    col_dict[str(i)] = f'alc{i}'

col_dict = {str(i) : f'alc{i}' for i in range(1960,2022)}

alcohol_wide.rename(columns =col_dict, inplace=True)

#drop_list=[*range(1960,2000)]
#drop_list.append(2021)

#alcohol_wide.drop(drop_list, axis=1, inplace=True)

# figure (alcohol)
def plot(df, country):
    I=df['country']==country
    ax=df.loc[I,:].plot(x='year', y='alcohol_sale', legend=False)

widgets.interact(plot, 
    df = widgets.fixed(alcohol),
    country = widgets.Dropdown(description='country', 
                                    options=alcohol.country.unique(), 
                                    value='AUS')
);

# figure (life_exp_65)
def plot(df, country):
    I=df['country']==country
    ax=df.loc[I,:].plot(x='year', y='exp_years', legend=False)

widgets.interact(plot, 
    df = widgets.fixed(life_exp_65_wide),
    country = widgets.Dropdown(description='country', 
                                    options=life_exp_65_wide.country.unique(), 
                                    value='AUS')
);

#life_exp_65_long = pd.melt(life_exp_65_wide, id_vars='country',value_vars='year')

# figure (overobese)
def plot(df, country):
    I=df['country']==country
    ax=df.loc[I,:].plot(x='year', y='share_obese', legend=False)

widgets.interact(plot, 
    df = widgets.fixed(overobese),
    country = widgets.Dropdown(description='country', 
                                    options=overobese.country.unique(), 
                                    value='AUS')
);

# figure (smokers)
def plot(df, country):
    I=df['country']==country
    ax=df.loc[I,:].plot(x='year', y='smoking', legend=False)

widgets.interact(plot, 
    df = widgets.fixed(smokers),
    country = widgets.Dropdown(description='country', 
                                    options=smokers.country.unique(), 
                                    value='AUS')
);

# figure (social_support)
def plot(df, country):
    I=df['country']==country
    ax=df.loc[I,:].plot(x='year', y='share_support', legend=False)

widgets.interact(plot, 
    df = widgets.fixed(social_support),
    country = widgets.Dropdown(description='country', 
                                    options=social_support.country.unique(), 
                                    value='AUS')
);

### MODELPROJECT

In [ ]:
# create a list of values for d
d_vec = [0,0.2,0.4,0.6,0.8]

# create an empty list to store the results
k_ss_list = []
y_ss_list = []
relative_y_ss_list = []

for d in d_vec:
    model.val.d = d
    k_ss, y_ss = model.solve_num_ss()
    k_ss_list.append(k_ss)
    y_ss_list.append(y_ss)
    rel_y_ss = y_ss/y_ss_list[0]*100
    relative_y_ss_list.append(rel_y_ss)

# define dropdown menu
d_dropdown = widgets.Dropdown(
    options=list(d_vec),
    description='Select d:',
    value=d_vec[0])

# define a function to print the text for dropdown
def print_text_dropdown(d):
    index = list(d_vec).index(d)
    print(f'd = {d:.1f}: \n Steady state for k is {k_ss_list[index]:.1f} \n steady state for y is {y_ss_list[index]:.1f} \n Steady state output per worker relative to a world without climate change is {relative_y_ss_list[index]:.1f}% \n')

# create an event handler to call the print_text function when the value of the dropdown menu changes
def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print_text_dropdown(change['new'])

d_dropdown.observe(on_dropdown_change)

# show dropdown menu
display(d_dropdown)

In [ ]:
# calling model parameters 
val = model.val

# define the transition equation with the production function integrated
kt1 = lambda k:(val.s*(1-val.d)*k**val.alpha + (1-val.delta)*k)/((1+val.g)*(1+val.n))

# generate a range of values for kt1
kt1_range = np.linspace(0, 10, 100)

# create a plot of the transition equation function
plt.plot(kt1_range, kt1(kt1_range))

# plot a line with slope 1 to represent the 45-degree line
plt.plot([0, 8], [0, 8], color='r')

# set the x and y limits
plt.xlim(0, 8)
plt.ylim(0, 8)

# label the axes
plt.xlabel(r'$\tilde k_t$')
plt.ylabel(r'$\tilde k_{t+1}$')

# set title
plt.title('Phase diagram')

# show the plot
plt.show()